## Homework: Evaluation and Monitoring

In this homework, we'll evaluate the quality of our RAG system.


## Getting the data

Let's start by getting the dataset. We will use the data we generated in the module.

In particular, we'll evaluate the quality of our RAG system
with [gpt-4o-mini](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv)


Read it:

```python
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
```

We will use only the first 300 documents:


```python
df = df.iloc[:300]

In [1]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]

In [2]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [3]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





## Q1. Getting the embeddings model

Now, get the embeddings model `multi-qa-mpnet-base-dot-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

> Note: this is not the same model as in HW3

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embeddings for the first LLM answer:

```python
answer_llm = df.iloc[0].answer_llm
```

What's the first value of the resulting vector?

* **-0.42**
* -0.22
* -0.02
* 0.21

In [4]:
answer_llm = df.iloc[0].answer_llm
embedding_model.encode(answer_llm)[0]

-0.42244658

## Q2. Computing the dot product


Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the `evaluations` list

What's the 75% percentile of the score?

* 21.67
* **31.67**
* 41.67
* 51.67

In [5]:
from tqdm import tqdm
import numpy as np

evaluations = []
for ind, row in tqdm(df.iterrows()):

    v_llm = embedding_model.encode(row.answer_llm)
    v_orig = embedding_model.encode(row.answer_orig)
    evaluations.append(v_llm.dot(v_orig))

300it [00:38,  7.77it/s]


In [6]:
df['dot_product'] = evaluations
df.dot_product.describe()

count    300.000000
mean      27.495993
std        6.384743
min        4.547927
25%       24.307846
50%       28.336858
75%       31.674307
max       39.476017
Name: dot_product, dtype: float64

## Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we 

* Compute the norm of a vector
* Divide each element by this norm

So, for vector `v`, it'll be `v / ||v||`

In numpy, this is how you do it:

```python
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Let's put it into a function and then compute dot product 
between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

* 0.63
* 0.73
* **0.83**
* 0.93

In [7]:
cosine_scores = []
for ind, row in tqdm(df.iterrows()):

    v_llm = embedding_model.encode(row.answer_llm)
    v_llm = v_llm / np.sqrt((v_llm * v_llm).sum())
    v_orig = embedding_model.encode(row.answer_orig)
    v_orig = v_orig / np.sqrt((v_orig * v_orig).sum())
    cosine_scores.append(v_llm.dot(v_orig))

300it [00:11, 25.49it/s]


In [8]:
df['cosine_scores'] = cosine_scores
df.cosine_scores.describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine_scores, dtype: float64

## Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.  

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```

(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

What's the F score for `rouge-1`?

- 0.35
- **0.45**
- 0.55
- 0.65

In [9]:
from rouge import Rouge
rouge_scorer = Rouge()
r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
r.document, scores['rouge-1']['f']

('5170565b', 0.45454544954545456)

## Q5. Average rouge score

Let's compute the average F-score between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4

- **0.35**
- 0.45
- 0.55
- 0.65

In [10]:
np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])

0.35490034990035496

## Q6. Average rouge score for all the data points

Now let's compute the F-score for all the records and create a dataframe from them.

What's the average F-score in `rouge_2` across all the records?

- 0.10
- **0.20**
- 0.30
- 0.40

In [11]:
rouge_2_f_scores = []
for ind, row in tqdm(df.iterrows()):
    scores = rouge_scorer.get_scores(row.answer_llm, row.answer_orig)[0]
    rouge_2_f_scores.append(scores['rouge-2']['f'])

300it [00:00, 610.18it/s]


In [12]:
np.mean(rouge_2_f_scores)

0.20696501983423318